# Azure AI Agent service - MCP support

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol

Supported regions: westus, westus2, uaenorth, southindia and switzerlandnorth

Install required packages:

`pip install gradio python-dotenv azure-identity azure-ai-projects>=1.0.0b12 azure-ai-agents>=1.1.0b4 --pre`

In [1]:
import os
import json
import re
import sys
import time

from azure.ai.agents.models import ListSortOrder, MessageTextContent, McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

## 1. Settings

In [ ]:
load_dotenv()

# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]
model_name = os.environ["MODEL_DEPLOYMENT_NAME"]

In [5]:
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT,
                                 credential=DefaultAzureCredential())

# Convenience handle for the Agents sub‑client
agents_client = project_client.agents

## 2. Example

In [6]:
# With this prefix 'https://gitmcp.io/' you can instantly create a remote RCP server from any GitHub repo
github_url = "https://github.com/Azure-Samples/ai-foundry-agents-samples"

get_substr = re.search(r'\.com/(.*)', github_url)
substr = get_substr.group(1)
mcp_server_url = f"https://gitmcp.io/{substr}"

mcp_server_url

'https://gitmcp.io/Azure-Samples/ai-foundry-agents-samples'

In [7]:
prompt = "Any python code for a mcp server with Azure AI Agent?"

In [8]:
project_client = AIProjectClient(endpoint=project_endpoint,
                                 credential=DefaultAzureCredential())

# MCP tool definition
mcp_tool = McpTool(
    server_label="github",
    server_url=mcp_server_url
    #allowed_tools=[],          # Optional initial allow‑list
)

# Agent creation
agent = agents_client.create_agent(
    model=model_name,
    name="agent-mcp",
    instructions="You are an AI agent. Use the tools provided to answer the user's questions. Be sure to cite your sources and answer in details.",
    tools=mcp_tool.definitions
)
print(f"🎉 Created agent, agent ID: {agent.id}")
print(f"✨ MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

# Thread creation
thread = agents_client.threads.create()
print(f"🧵 Created thread, thread ID: {thread.id}")

# Message creation
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)
print(f"💬 Created message, message ID: {message.id}")

mcp_tool.set_approval_mode("never")          # Disable human approval

# Run
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(2)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"⏳ Run status: {run.status}")
if run.status == "failed":
    print(f"❌ Run error: {run.last_error}")

# Run steps
run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
print()

for step in run_steps:
    print(f"🔄 Run step: {step.id}, status: {step.status}, type: {step.type}")
    if step.type == "tool_calls":
        print(f"🛠️ Tool call details:")
        for tool_call in step.step_details.tool_calls:
            print(json.dumps(tool_call.as_dict(), indent=5))


🎉 Created agent, agent ID: asst_XhAxDbbiXLUgB2WuyBZDzteD
✨ MCP Server: github at https://gitmcp.io/Azure-Samples/ai-foundry-agents-samples
🧵 Created thread, thread ID: thread_FMhuDOE273T6GmDSd8GFs2tZ
💬 Created message, message ID: msg_EsS7hZEd2YOLhT2hZMFR5a1o
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.COMPLETED

🔄 Run step: step_C6x5DoBcMOKU77mM6IlsOIOe, status: RunStepStatus.COMPLETED, type: RunStepType

In [9]:
messages = agents_client.messages.list(thread_id=thread.id,
                                               order=ListSortOrder.ASCENDING)

for item in messages:
    last_message_content = item.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"{item.role}: {last_message_content.text.value}")

MessageRole.USER: Any python code for a mcp server with Azure AI Agent?
MessageRole.AGENT: Yes, there is Python sample code for an MCP server that can be used with the Azure AI Agent, as found in the [Azure-Samples/ai-foundry-agents-samples](https://github.com/Azure-Samples/ai-foundry-agents-samples) repository.

### MCP Server Python Example

A minimal MCP server implemented in Python can be found here:
- [`main.py` for MCP server](https://github.com/Azure-Samples/ai-foundry-agents-samples/blob/main/examples/mcp/streamable-http/mcp-server/main.py)

```python
def main():
    print("Hello from mcp-server!")

if __name__ == "__main__":
    main()
```
> This is just the template; in real use, you implement API endpoints that serve MCP Tool functionality.

### Azure AI Agent Python Example

The repository contains a full-featured example of communicating with MCP Servers via an Azure AI Agent, including building an agent that uses the MCP tool. Here is the core of the agent code connecting

In [ ]:
##Clean Up: Delete Agent

In [ ]:
agents_client.delete_agent(agent.id)
print(f"Deleted agent, agent ID: {agent.id}")